Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Imports

In [ ]:
import random
import glob
from csv import writer
import csv
import os
import shutil
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import time
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import Image as ig

Count of Train and Validation Sets

In [ ]:
import os
train='/content/gdrive/My Drive/Mobisy/trainingSet/val/'
valid='/content/gdrive/My Drive/Mobisy/trainingSet/train/'
sum_t=0
sum_v=0
for j in range(0,10):
  sum_t=sum_t+len([entry for entry in os.listdir(train+str(j)) if os.path.isfile(os.path.join(train+str(j), entry))])
  sum_v=sum_v+len([entry for entry in os.listdir(valid+str(j)) if os.path.isfile(os.path.join(valid+str(j), entry))])
print(sum_t,sum_v)

0 22599


In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Data Transforms

In [ ]:
data_path = '/content/gdrive/My Drive/Mobisy/trainingSet/'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Grayscale(1),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(1),
    ]),
}

In [ ]:
BATCH_SIZE = 64


In [ ]:

image_datasets = {x: datasets.ImageFolder(root=os.path.join(data_path, x),transform=data_transforms[x]) for x in ['train', 'val']}
print(image_datasets)
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['train', 'val']}


{'train': Dataset ImageFolder
    Number of datapoints: 23239
    Root location: /content/gdrive/My Drive/Mobisy/trainingSet/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               Grayscale(num_output_channels=1)
               ToTensor()
           ), 'val': Dataset ImageFolder
    Number of datapoints: 8431
    Root location: /content/gdrive/My Drive/Mobisy/trainingSet/val
    StandardTransform
Transform: Compose(
               ToTensor()
               Grayscale(num_output_channels=1)
           )}


Load dataset and apply dataloaders

In [ ]:
target = image_datasets['train'].classes

In [ ]:
dataset_sizes = {x:len(image_datasets[x]) for x in ['train', 'val']}
dataset_sizes['val']

8431

In [ ]:
def create_torch_tensor(input):
  second_input = torch.as_tensor(input).to(device)
  second_input=second_input.float()
  second_output = torch.sum(second_input, axis=-1).reshape(-1, 1)  # (N, C)
  return second_input,second_output

Model 1 - Classification Model

In [ ]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1) 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)                                  
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=2)                               
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)                                  
        self.fc1 = nn.Linear(8*11*11, 120)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))         # output: 26*26 -> # output: 25*25
        x = self.pool2(self.relu(self.conv2(x)))         # output:  12*12 -> 11*11
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net1 = Model1()

Model 2 - Summation of 2 numbers

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()                             
        self.fc1 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

net2 = Model2()

Building Methods to train model 1

In [ ]:
class Methods:
    def __init__(self):
        self.model1 = None
        
    def train(self, model1,model2, epochs1,epochs2, optimizer1,optimizer2, criterion1,criterion2):
        self.model1 = model1
        self.model2 = model2
        start = time.time()
        for epoch in (range(epochs1)):  
            print(f'Epoch: {epoch+1}')
            for mode in ['train', 'val']:
                running_loss_m1 = 0.0
                running_acc_m1 = 0.0
                running_loss_m2 = 0.0
                running_acc_m2 = 0.0
                for i, data in tqdm(enumerate(dataloaders[mode])):
                    inputs, labels = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    inputs = inputs.float()
                    optimizer1.zero_grad()
                    outputs = self.model1(inputs)
                    loss1 = criterion1(outputs, labels)

                    if mode=='train':
                      
                        labels_in_process=labels.tolist()
                        input_data_processed=[]
                        for m in range(0,len(labels_in_process)):
                          input_data_processed.append([labels_in_process[m],random.randint(0,9)])
                        second_input,second_output=create_torch_tensor(input_data_processed)
                        for second_epoch in range(epochs2):
                          y_pred = model2(second_input) 
                          loss2 = criterion2(y_pred, second_output)  
                          loss2.backward()  
                          optimizer2.step() 
                          optimizer2.zero_grad() 
                          end = time.time()
                          training_time = end-start
                          print(f'Training Completed in: {training_time//60} min {training_time%60:.2f} sec')
                        loss1.backward()
                        
                        optimizer1.step()

               
                    running_loss_m1 += loss1.item()
                    running_loss_m2 += loss2.item()
                
                    outputs = torch.log_softmax(outputs, dim=1)
                    
                    max_vals, max_idx = torch.max(outputs, 1)
    
                    check = torch.sum(max_idx==labels)
                    running_acc_m1 += check
                    
                if mode=='train':
                    print(f'Training Loss model 1: {running_loss_m1:.3f} Training Accuracy: {(100*running_acc_m1/dataset_sizes[mode]):.2f}%')
                    print(f'Training Loss model 2: {running_loss_m2:.3f}')
                else:
                    print(f'Validation Loss: {running_loss_m1:.3f} Validation Accuracy: {(100*running_acc_m1/dataset_sizes[mode]):.2f}%')
                    print(f'Validation Loss: {running_loss_m2:.3f}')
                
            print(f'-----------------------------------')
            end = time.time()
            training_time = end-start
            print(f'Training Completed in: {training_time//60} min {training_time%60:.2f} sec')
            break

        end = time.time()
        training_time = end-start
        print(f'Training Completed in: {training_time//60} min {training_time%60:.2f} sec')
        print('Finished Training')
        return self.model1,self.model2


Loss1 and Optimizer1


In [ ]:
criterion_1 = nn.CrossEntropyLoss()
optimizer_1 = optim.SGD(net1.parameters(), lr=0.001, momentum=0.9)

Loss2 and Optimizer2


In [ ]:
criterion_2 = nn.MSELoss()  # loss function
optimizer_2 = optim.Adam(net2.parameters(), lr=1e-2)  # optimizer

Train Model 1

In [ ]:
EPOCHS_1 = 10
method_1 = Methods()

In [ ]:
EPOCHS_2 = 10


In [ ]:
trained_model_1,trained_model_2 = method_1.train(net1.to(device),net2.to(device), EPOCHS_1,EPOCHS_2, optimizer_1, optimizer_2,criterion_1 , criterion_2)

Epoch: 1


0it [00:00, ?it/s]

KeyboardInterrupt: ignored

Save Model 1

In [ ]:
FILE_PATH_1 = '/content/gdrive/My Drive/Mobisy/trainedModels/mnist_image_classification_model.pt'
torch.save(trained_model_1.state_dict(), FILE_PATH_1)

Train Model 2

In [ ]:
# second_input,second_output=create_torch_tensor(input_data_processed)

In [ ]:
# trained_model_2 = method_2.train(net2.to(device),second_input,second_output ,EPOCHS_2, optimizer_2, criterion_2)


Save Model 2

In [ ]:
FILE_PATH_2 = '/content/gdrive/My Drive/Mobisy/trainedModels/summation_model.pt'
torch.save(trained_model_2.state_dict(), FILE_PATH_2)

Load Model 1

In [ ]:
net1.load_state_dict(torch.load(FILE_PATH_1))
net1.eval()

Load Model 2

In [ ]:
net2.load_state_dict(torch.load(FILE_PATH_2))
net2.eval()

Test Image

In [ ]:
def testImage(model, PATH):
    image = Image.open(PATH)
    model = model.to(device)
    transform = transforms.ToTensor()
    img_tensor = transform(image)
    img_tensor = torch.unsqueeze(img_tensor, 0)
    model.to(device)
    img_tensor = img_tensor.to(device)
    img_tensor = img_tensor.float()
    pred = model(img_tensor)
    check = F.softmax(pred, dim=1)
    idx = torch.argmax(check, dim=1)
    
    return target[idx]

Test Summation

In [ ]:
def testSummation(model,input):
    pred=model(input)
    return pred.item()


Load Test Images

In [ ]:
gdrive_testimages_path='/content/gdrive/My Drive/Mobisy/testSet/testSet'
test_images = glob.glob(gdrive_testimages_path + '/*.jpg') 

Final Result

In [ ]:
for i in test_images:
  image_output=int(testImage(net1,i))
  second_model_input=[[image_output,random.randint(0,9)]]
  second_input = torch.as_tensor(second_model_input).to(device)
  second_input=second_input.float()
  summation=testSummation(net2,second_input)
  print(i,image_output,second_model_input,summation)
